In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [ ]:
import pathlib
dataset_url = "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/3fmjm7ncc6-2.zip"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')

In [ ]:
from zipfile import ZipFile
with ZipFile ("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset.zip",'r') as Soybean_Data:
  Soybean_Data.extractall(
        path="/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/")


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('accuracy')>.97):
      print("\nReached 97% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
broadleaf_images = os.listdir("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/broadleaf")
grass_images = os.listdir("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/grass")
soil_images = os.listdir("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soil")
soybean_images = os.listdir("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soybean")


print (f'There are {len(broadleaf_images)} broadleaf images.')
print (f'There are {len(grass_images)} grass images.')
print (f'There are {len(soil_images)} soil images.')
print (f'There are {len(soybean_images)} soybean images.')

There are 1191 broadleaf images.
There are 3520 grass images.
There are 3249 soil images.
There are 7376 soybean images.


In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
#Converts .tif files to .jpg

import os
from PIL import Image

os.chdir("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset")

weed_detection = os.getcwd()
for root, dirs, files in os.walk(weed_detection, topdown=False):
    for name in files:
        print(os.path.join(root, name))
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".tif":
            if os.path.isfile(os.path.splitext(os.path.join(root, name))[0] + ".jpg"):
                print ("A jpeg file already exists for %s" % name)
            # If a jpeg is *NOT* present, create one from the tiff.
            else:
                outfile = os.path.splitext(os.path.join(root, name))[0] + ".jpg"
                try:
                    im = Image.open(os.path.join(root, name))
                    print ("Generating jpeg for %s" % name)
                    im.thumbnail(im.size)
                    im.save(outfile, "JPEG", quality=100)
                except Exception:
                    print ("There was an error")

Streaming output truncated to the last 5000 lines.
/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soybean/3414.tif
Generating jpeg for 3414.tif
/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soybean/2850.tif
Generating jpeg for 2850.tif
/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soybean/5841.tif
Generating jpeg for 5841.tif
/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soybean/4572.tif
Generating jpeg for 4572.tif
/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soybean/6736.tif
Generating jpeg for 6736.tif
/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soybean/2899.tif
Generating jpeg for 2899.tif
/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soybean/2097.tif
Generating jpeg for 2097.tif
/root/.keras/datasets/Dat

In [ ]:
#Deletes the .tif files in the directory for cleanup.

weed_detection ='/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset'

for root, dirs, files in os.walk(weed_detection):
  for name in files:
    if name.endswith((".tif")):
      os.remove(os.path.join(root, name))

In [ ]:
#Confirm deletion of .tif files

broadleaf_images = os.listdir("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/broadleaf")
grass_images = os.listdir("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/grass")
soil_images = os.listdir("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soil")
soybean_images = os.listdir("/root/.keras/datasets/Data for Weed Detection in Soybean Crops Using ConvNets/dataset/soybean")


print (f'There are {len(broadleaf_images)} broadleaf images.')
print (f'There are {len(grass_images)} grass images.')
print (f'There are {len(soil_images)} soil images.')
print (f'There are {len(soybean_images)} soybean images.')

There are 1191 broadleaf images.
There are 3520 grass images.
There are 3249 soil images.
There are 7376 soybean images.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=128, kernel_size=3,
                           strides=1, padding="causal",
                           activation="relu",
                           input_shape=[300, 300, 3]),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
       optimizer=RMSprop(lr=0.001),
       metrics=['accuracy'])

optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.5)

model.compile(loss="mse", optimizer=optimizer)

history = model.fit(dataset, epochs=100, verbose=1)

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
forecast = model_forecast(model, series[..., np.newaxis], window_size)

In [ ]:
results = forecast[split_time - window_size:-1, -1, 0]